In [17]:
from random import choice
from collections import defaultdict

def ConstructGraphFromFile(file):
    with open(file) as f:
        graph = {}
        for line in f:
            edge = line.strip().split(': ')
            graph[edge[0]] = edge[1].split(' ')
    return graph

def ConstructGraph(file):
    graph = {}
    edge = file.strip().split(': ')
    graph[edge[0]] = edge[1].split(' ')
    return graph

def EulerianCycle(graph, node):
    cycle = [node]
    while graph[node]:
        working = EulerianCycle(graph, graph[node].pop())
        cycle = cycle[:1] + working + cycle[1:]
    return cycle

#Todo: Implement recursive method. 
def EulerianPath(graph):
    diff = {}
    for key, values in graph.items():
        if key not in diff:
            diff[key] = len(values)
        else:
            diff[key] += len(values)
        for value in values:
            if value not in diff:
                diff[value] = -1
            else:
                diff[value] -= 1
    keys = [node for node, dif in diff.items() if dif == -1][0]
    values = [node for node, dif in diff.items() if dif == 1][0]
    if keys in graph:
        graph[keys].append(values)
    else:
        graph[keys] = [values]
    
    initial = list(graph)[0]
    cycle = EulerianCycle(graph,initial )
    index = 0
    while (True):
        if cycle[index] == keys and cycle[index + 1] == values:
            break
        index += 1
    return cycle[index + 1:] + cycle[1:index + 1]

In [2]:
def DeBruijnGraphPairedReads(reads):
    graph = defaultdict(list)
    for read in reads:
        graph[(read[0][:-1], read[1][:-1])].append((read[0][1:], read[1][1:]))
    return graph

In [3]:
def PathToGenome(reads):
    path = reads[0]
    for read in reads[1:]:
        n = len(read)
        path += read[n-1:]
    return path

In [16]:
from collections import defaultdict
def string_spelled_by_g_p(gappedPatterns, k, d):
    firstPatterns = []
    secondPatterns = []
    for kDmers in gappedPatterns:
        firstPatterns.append(kDmers[0])
        secondPatterns.append(kDmers[1])
    DeBruijn = DeBruijnGraphPairedReads(gappedPatterns)

    path = EulerianPath(DeBruijn)

    fPatterns = []
    sPatterns = []
    for step in path:
        fPatterns.append(step.split('|')[0])
        sPatterns.append(step.split('|')[1])

    prefixString = PathToGenome(fPatterns)
    suffixString = PathToGenome(sPatterns)

    prefixEnder = k + d + 1
    for i in range(prefixEnder, len(prefixString) + 1):
        suffixOverlap = i - k - d
        if prefixString[i] != suffixString[suffixOverlap]:
            return 'There is no string spelled by the gapped patterns'
        return prefixString + suffixString[-(prefixEnder - 1):]

In [5]:
def string_reconstruction_read_pairs(k, d, paired_reads):
    adj_list = DeBruijnGraphPairedReads(paired_reads)
    path = EulerianPath(adj_list)
    return string_spelled_by_g_p(path, k - 1, d)


In [20]:
import io
with io.open("StringReconstructionReadPairs\inputs\input_1.txt", mode="r", encoding="utf-8") as f:
    working = f.readline()
    k, d = [int(x) for x in working.split()]
    print(k,d)
    w1 = f.readline().split(': ')    
    ans = string_reconstruction_read_pairs(k, d, w1)
    print(ans)

4 2


IndexError: list index out of range